In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 3.5MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [99]:
cd "drive/My Drive/Colab Notebooks/dw_matrix"

[Errno 2] No such file or directory: 'drive/My Drive/Colab Notebooks/dw_matrix'
/content/drive/My Drive/Colab Notebooks/dw_matrix


In [100]:
ls data/

men_shoes.csv


In [0]:
df=pd.read_csv('data/men_shoes.csv',low_memory=False)

In [0]:
def run_model(feats,model=DecisionTreeRegressor(max_depth=5)):
    X=df[feats].values
    y=df['prices_amountmin'].values

    scores=cross_val_score(model,X,y,scoring='neg_mean_absolute_error')
    return np.mean(scores),np.std(scores)

In [103]:
df['brand_cat']=df['brand'].map(lambda x : str(x).lower()).factorize()[0]
run_model(['brand_cat'],model)

(-57.34154247268748, 4.15865946146927)

In [104]:
model=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [105]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
test={'key':'value'}

In [0]:
def parse_features(x):
  if str(x)=='nan':return[]
  return literal_eval(x.replace('\\"','"',))

df['features_parsed'] =df['features'].map(parse_features)

In [107]:
df['features_parsed'] .head()

0    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
1    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
2    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
3    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
4    [{'key': 'Gender', 'value': ['Men']}, {'key': ...
Name: features_parsed, dtype: object

In [0]:
def parse_features(x):
  output_dict={}
  if str(x)=='nan':return output_dict
  features=literal_eval(x.replace('\\"','"',))
  for item in features:
    key=item['key'].lower().strip()
    value=item['value'][0].lower().strip()
    output_dict[key]=value
  return output_dict

df['features_parsed'] =df['features'].map(parse_features)

In [109]:
keys=set()

df['features_parsed'] .map(lambda x : keys.update(x.keys()))

len(keys)

476

In [110]:
def get_name_feat(key):
  return 'feat_'+key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)]=df.features_parsed.map(lambda feats:feats[key] if key in feats else np.nan)


In [111]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_display technology', 'feat_product id', 'feat_garment care',
       'feat_high visibility (ansi compliant)', 'feat_lens material',
       'feat_battery type', 'feat_multi pack indicator',
       'feat_safety features', 'feat_band manufacturer', 'feat_waist size'],
      dtype='object', length=526)

In [0]:
keys_stat={}
for key in keys:
  keys_stat[key]=df[False==df[get_name_feat(key)].isnull()].shape[0]/df.shape[0]*100


In [113]:
{k:v for k,v in keys_stat.items() if v>30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat']=df['feat_brand'].factorize()[0]
df['feat_color_cat']=df['feat_color'].factorize()[0]
df['feat_gender_cat']=df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat']=df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat']=df['feat_material'].factorize()[0]
df['feat_sport_cat']=df['feat_sport'].factorize()[0]
df['feat_style_cat']=df['feat_style'].factorize()[0]

In [115]:
df['brand']=df['brand'].map(lambda x: str(x).lower().strip())
df[df.brand==df.feat_brand].shape

(8846, 533)

In [116]:
model=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [132]:
feats=['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat',
       'feat_manufacturer part number_cat','feat_material_cat','feat_sport_cat','feat_style_cat']
model=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
run_model(feats,model)

(-57.2529051143538, 4.297649682499428)

In [118]:
X=df[feats].values
y=df['prices_amountmin'].values
m=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)

perm=PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

Weight,Feature
0.2524 ± 0.0075,brand_cat
0.1018 ± 0.0109,feat_material_cat
0.0451 ± 0.0015,feat_gender_cat
0.0200 ± 0.0007,feat_brand_cat
0.0070 ± 0.0012,feat_color_cat
0.0068 ± 0.0010,feat_style_cat
0.0006 ± 0.0001,feat_manufacturer part number_cat
0.0003 ± 0.0000,feat_sport_cat


In [119]:
feats=['brand_cat','feat_brand_cat','feat_color_cat','feat_gender_cat','feat_material_cat','feat_style_cat']
model=RandomForestRegressor(max_depth=5,n_estimators=100)
run_model(feats,model)

(-57.07030144381389, 4.393388413165106)

In [120]:
df.brand.value_counts(normalize=True)

nike                               0.097210
puma                               0.033315
ralph lauren                       0.028775
vans                               0.021116
new balance                        0.020295
                                     ...   
jbw                                0.000055
zorrie                             0.000055
fxr                                0.000055
walleva (not an oakley product)    0.000055
us toy                             0.000055
Name: brand, Length: 1732, dtype: float64

In [121]:
df[df['brand']=='nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].map(lambda x: str(x).lower()).factorize()[0]

In [123]:
feats_cat = {k:v for k,v in keys_stat.items() if v>10}
feats_cat = [get_name_feat(x) + '_cat' for x in feats_cat] + ['brand_cat']
feats_cat

['feat_shoe width_cat',
 'feat_color_cat',
 'feat_age group_cat',
 'feat_style_cat',
 'feat_casual & dress shoe style_cat',
 'feat_model_cat',
 'feat_assembled product dimensions (l x w x h)_cat',
 'feat_material_cat',
 'feat_shoe category_cat',
 'feat_shoe size_cat',
 'feat_brand_cat',
 'feat_manufacturer part number_cat',
 'feat_size_cat',
 'feat_gender_cat',
 'feat_condition_cat',
 'brand_cat']

In [129]:
X=df[feats_cat].values
y=df['prices_amountmin'].values
m=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)

print(results)
perm=PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats_cat)

(-57.20571776552849, 4.4391041957788575)


Weight,Feature
0.2628 ± 0.0080,brand_cat
0.1078 ± 0.0118,feat_material_cat
0.0463 ± 0.0029,feat_gender_cat
0.0154 ± 0.0010,feat_shoe category_cat
0.0145 ± 0.0021,feat_brand_cat
0.0055 ± 0.0010,feat_style_cat
0.0045 ± 0.0005,feat_casual & dress shoe style_cat
0.0021 ± 0.0004,feat_shoe size_cat
0.0018 ± 0.0004,feat_condition_cat
0.0016 ± 0.0002,feat_size_cat


In [162]:
feats=['brand_cat','feat_brand_cat','feat_gender_cat','feat_material_cat',
       'feat_sport_cat','feat_style_cat','feat_casual & dress shoe style_cat','feat_shoe size_cat',
       'feat_age group_cat']
model=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
results=run_model(feats,model)
results

(-57.10902955709615, 4.258381152967017)

In [163]:
X=df[feats].values
y=df['prices_amountmin'].values
m=RandomForestRegressor(max_depth=5,n_estimators=100,random_state=0)
m.fit(X,y)

print(results)
perm=PermutationImportance(m,random_state=1).fit(X,y);
eli5.show_weights(perm,feature_names=feats)

(-57.10902955709615, 4.258381152967017)


Weight,Feature
0.2604 ± 0.0127,brand_cat
0.1104 ± 0.0044,feat_material_cat
0.0445 ± 0.0018,feat_gender_cat
0.0198 ± 0.0012,feat_brand_cat
0.0062 ± 0.0009,feat_style_cat
0.0025 ± 0.0004,feat_shoe size_cat
0.0016 ± 0.0002,feat_casual & dress shoe style_cat
0.0011 ± 0.0007,feat_age group_cat
0.0000 ± 0.0000,feat_sport_cat


In [0]:
!git add matrix_one/day5.ipynb